# Intro to CUDA (using Python!)

In [31]:
import numpy as np
import cupy as cp

# A quick set of polls

Go to the link below and fill out the poll


# Data Types

## Static vs Dynamic Typing 

https://twitter.com/01k/status/1067788059989684224
<div>
<img src="./img/static_vs_dynamic.png" width="400"/>
</div>

## Duck Typing

https://stackoverflow.com/questions/4205130/what-is-duck-typing

<div>
<img src="./img/duck_typing.png" width="400"/>
</div>

pseudo-code example (don't run the next cell!)

In [ ]:
A = car()
A.drive() #works!

B = semi_truck()
B.drive() #works!

C = golf_club()
C.drive() #works!

D = coffee()
D.drive() #fails!!

## Data Type Demonstration in Python

In [23]:
%load_ext nb_mypy

Version 1.0.5


In [91]:
%nb_mypy On

In [92]:
def add_vectors(x1,x2):
    y = np.zeros_like(x1)
    for i in range(x1.shape[0]):
        for j in range(x1.shape[1]):
            y[i,j] = x1[i,j] + x2[i,j]
    return y

In [93]:
x1 = np.arange(25).reshape(5,5)
x1

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24]])

In [94]:
x2 = np.arange(25,50).reshape(5,5)
x2

array([[25, 26, 27, 28, 29],
       [30, 31, 32, 33, 34],
       [35, 36, 37, 38, 39],
       [40, 41, 42, 43, 44],
       [45, 46, 47, 48, 49]])

In [95]:
add_vectors(x1,x2)

array([[25, 27, 29, 31, 33],
       [35, 37, 39, 41, 43],
       [45, 47, 49, 51, 53],
       [55, 57, 59, 61, 63],
       [65, 67, 69, 71, 73]])

In [52]:
x3 = list(x2)
add_vectors(x1,x3)

AttributeError: 'list' object has no attribute 'shape'

In [39]:
def add_vectors_mypy(x1:np.typing.NDArray,x2:np.typing.NDArray)->np.typing.NDArray:
    y = np.zeros_like(x1)
    for i in range(x1.shape[0]):
        for j in range(x1.shape[1]):
            y[i,j] = x1[i,j] + x2[i,j]
    return y

In [53]:
add_vectors_mypy(x1,x2)

array([[25, 27, 29, 31, 33],
       [35, 37, 39, 41, 43],
       [45, 47, 49, 51, 53],
       [55, 57, 59, 61, 63],
       [65, 67, 69, 71, 73]])

In [58]:
add_vectors_mypy(x1,x3)

<cell>1: error: Argument 2 to "add_vectors_mypy" has incompatible type "List[Any]"; expected "ndarray[Any, dtype[Any]]"  [arg-type]


AttributeError: 'list' object has no attribute 'shape'

In [60]:
%nb_mypy Off

# CUDA Basics

## Grids, Blocks, and Threads

https://cs.calvin.edu/courses/cs/374/CUDA/CUDA-Thread-Indexing-Cheatsheet.pdf

https://en.wikipedia.org/wiki/Thread_block_%28CUDA_programming%29

<div>
<img src="./img/threads_blocks2.png" width="500"/>
</div>

## Intro to CuPy: CUDA/Python Interface

In [ ]:
x1 = cp.arange(25,dtype=cp.float32).reshape(5,5)
x1

array([[ 0.,  1.,  2.,  3.,  4.],
       [ 5.,  6.,  7.,  8.,  9.],
       [10., 11., 12., 13., 14.],
       [15., 16., 17., 18., 19.],
       [20., 21., 22., 23., 24.]], dtype=float32)

In [ ]:
x2 = cp.arange(25,50,dtype=cp.float32).reshape(5,5)
x2

array([[25., 26., 27., 28., 29.],
       [30., 31., 32., 33., 34.],
       [35., 36., 37., 38., 39.],
       [40., 41., 42., 43., 44.],
       [45., 46., 47., 48., 49.]], dtype=float32)

In [ ]:
y = x1+x2
y

array([[25., 27., 29., 31., 33.],
       [35., 37., 39., 41., 43.],
       [45., 47., 49., 51., 53.],
       [55., 57., 59., 61., 63.],
       [65., 67., 69., 71., 73.]], dtype=float32)

In [ ]:
x3 = cp.arange(10000,dtype=cp.float32).reshape(100,100)
y = x3@x3
y

Alternatives to cupy
- pycuda
- tensorflow
- pytorch
- numba

# CUDA Kernels: 1D Thread Grids

In [2]:
add_kernel = cp.RawKernel(r'''
extern "C" __global__
void cuda_add(const float* x1, const float* x2, float* y) {
    int tid = blockDim.x * blockIdx.x + threadIdx.x;
    y[tid] = x1[tid] + x2[tid];
}
''', 'cuda_add')

In [3]:
x1 = cp.arange(25,dtype=cp.float32).reshape(5,5)
x2 = cp.arange(25,50,dtype=cp.float32).reshape(5,5)

In [21]:
#%%timeit
y = cp.zeros((5,5),dtype=cp.float32)
add_kernel(
    (1,), # grid shape (number of blocks in each dimension)
    (1,), # block shape (number of threads in each dimension)
    (x1,x2,y)
)
y

array([[25.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.]], dtype=float32)

In [30]:
#%%timeit
y = cp.zeros((5,5),dtype=cp.float32)
add_kernel(
    (5,), # grid shape (number of blocks in each dimension)
    (1,), # block shape (number of threads in each dimension)
    (x1,x2,y)
)
y

array([[25., 27., 29., 31., 33.],
       [35., 37., 39., 41., 43.],
       [45., 47., 49., 51., 53.],
       [55., 57., 59., 61., 63.],
       [65., 67., 69., 71., 73.]], dtype=float32)

In [28]:
#%%timeit
y = cp.zeros((5,5),dtype=cp.float32)
add_kernel(
    (1,), # grid shape (number of blocks in each dimension)
    (5,), # block shape (number of threads in each dimension)
    (x1,x2,y)
)
y

array([[25., 27., 29., 31., 33.],
       [35., 37., 39., 41., 43.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.]], dtype=float32)

In [101]:
#%%timeit
y = cp.zeros((5,5),dtype=cp.float32)
add_kernel(
    (25,), # grid shape (number of blocks in each dimension)
    (1,), # block shape (number of threads in each dimension)
    (x1,x2,y)
)
y

array([[25., 27., 29., 31., 33.],
       [35., 37., 39., 41., 43.],
       [45., 47., 49., 51., 53.],
       [55., 57., 59., 61., 63.],
       [65., 67., 69., 71., 73.]], dtype=float32)

In [102]:
#%%timeit
y = cp.zeros((5,5),dtype=cp.float32)
add_kernel(
    (1,), # grid shape (number of blocks in each dimension)
    (25,), # block shape (number of threads in each dimension)
    (x1,x2,y)
)
y

array([[25., 27., 29., 31., 33.],
       [35., 37., 39., 41., 43.],
       [45., 47., 49., 51., 53.],
       [55., 57., 59., 61., 63.],
       [65., 67., 69., 71., 73.]], dtype=float32)

# CUDA Kernels: 2D Thread Grids

## A Quick Diversion: Flattened Arrays

In [36]:
z1 = np.arange(25).reshape(5,5)
z1

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24]])

In [37]:
z1_flat = z1.ravel()
z1_flat

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24])

In [40]:
i = 2
j = 3
z1[i,j]

13

In [42]:
N = z1.shape[0]
k = i*N+j
z1_flat[k]

13

## Back to CUDA

In [4]:
add_kernel_2D = cp.RawKernel(r'''
extern "C" __global__
void cuda_add_2D(const float* x1, const float* x2, float* y, int N) {
    int i = blockDim.y * blockIdx.y + threadIdx.y;
    int j = blockDim.x * blockIdx.x + threadIdx.x;
    
    int k = i*N+j;
    y[k] = x1[k] + x2[k];
    
}
''', 'cuda_add_2D')

In [16]:
y = cp.zeros((5,5),dtype=cp.float32)
add_kernel_2D(
    (1,1), # grid shape (number of blocks in each dimension)
    (1,1), # block shape (number of threads in each dimension)
    (x1,x2,y,x1.shape[0])
)
y

array([[25.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.]], dtype=float32)

In [43]:
y = cp.zeros((5,5),dtype=cp.float32)
add_kernel_2D(
    (1,1), # grid shape (number of blocks in each dimension)
    (5,5), # block shape (number of threads in each dimension)
    (x1,x2,y,x1.shape[0])
)
y

array([[25., 27., 29., 31., 33.],
       [35., 37., 39., 41., 43.],
       [45., 47., 49., 51., 53.],
       [55., 57., 59., 61., 63.],
       [ 0.,  0.,  0.,  0.,  0.]], dtype=float32)

In [44]:
y = cp.zeros((5,5),dtype=cp.float32)
add_kernel_2D(
    (5,5), # grid shape (number of blocks in each dimension)
    (1,1), # block shape (number of threads in each dimension)
    (x1,x2,y,x1.shape[0])
)
y

array([[25., 27., 29., 31., 33.],
       [35., 37., 39., 41., 43.],
       [45., 47., 49., 51., 53.],
       [55., 57., 59., 61., 63.],
       [65., 67., 69., 71., 73.]], dtype=float32)

# CUDA Kernels: Matrix Multiplication

In [52]:
mult_kernel = cp.RawKernel(r'''
extern "C" __global__
void cuda_mult(const float* x1, const float* x2, float* y, int N) {
    int ROW = blockDim.y * blockIdx.y + threadIdx.y;
    int COL = blockDim.x * blockIdx.x + threadIdx.x;
    
    float tmpSum = 0;

    if (ROW < N && COL < N) {
        // each thread computes one element of the block sub-matrix
        for (int i = 0; i < N; i++) {
            tmpSum += x1[ROW * N + i] * x2[i * N + COL];
        }
    }
    y[ROW * N + COL] = tmpSum;
    
}
''', 'cuda_mult')

In [55]:
x1 = cp.arange(25,dtype=cp.float32).reshape(5,5)
x2 = cp.arange(25,dtype=cp.float32).reshape(5,5)
y = cp.zeros((5,5),dtype=cp.float32)

In [56]:
#%%timeit
mult_kernel(
    (1,1), # grid shape (number of blocks in each dimension)
    (5,5), # block shape (number of threads in each dimension)
    (x1,x2,y,x1.shape[0])
)
y

5.44 µs ± 31.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Let's try to push this...

In [74]:
x1 = cp.arange(1000000,dtype=cp.float32).reshape(1000,1000)
x2 = cp.arange(1000000,dtype=cp.float32).reshape(1000,1000)
y = cp.zeros((1000,1000),dtype=cp.float32)

In [ ]:
%%timeit
mult_kernel(
    (100,100), # grid shape (number of blocks in each dimension)
    (100,100), # block shape (number of threads in each dimension)
    (x1,x2,y,x1.shape[0])
)
y

In [ ]:
add_kernel = cp.RawKernel(r'''
extern "C" __global__
void cuda_add(const float* x1, const float* x2, float* y) {
    int row = blockDim.y * blockIdx.y + threadIdx.x;
    y[tid] = x1[tid] + x2[tid];
}
''', 'cuda_add')